In [10]:
import torch
from models.cgan import GeneratorModel, DiscriminatorModel, ClassifierModel
import numpy as np
from torchvision import datasets, transforms
from matplotlib import pyplot as plt
from metric.niqe import niqe
from metric.fid import calculate_fid
from skimage.transform import rescale, resize
from imageio import imread
from collections import defaultdict
from tqdm import tqdm

In [11]:
g = GeneratorModel()
d = DiscriminatorModel()
c = ClassifierModel()
gsd, dsd = torch.load('ckpt/cgan.pt', map_location='cpu')
g.load_state_dict(gsd)
d.load_state_dict(dsd)

<All keys matched successfully>

In [ ]:
transform_fn = transforms.Compose([transforms.ToTensor(), transforms.CenterCrop(178), transforms.Resize(64)])
train_data = datasets.CelebA('./data', split='train', download=True, transform=transform_fn)

In [2]:
g = g.cuda()
d = d.cuda()

NameError: name 'g' is not defined

In [ ]:
score_dic = defaultdict(list)
for index in tqdm(range(10)):
    img, label = train_data[index]
    img  = img.unsqueeze(0).cuda()
    label = label.unsqueeze(0).cuda()
    f, axs = plt.subplots(5, label.size(1) // 5, figsize=(16, 10))
    axs = axs.flatten()
    o = np.copy(img.cpu().numpy())
    o.resize(200, 200)
    score_dic['niqe_ori'].append(niqe(np.copy(o)))
    for i in range(label.size(1)):
        label[0, i] = 1 - label[0, i]
        with torch.no_grad():
            o = g(img, label.cuda())
        o = np.moveaxis(o.squeeze().cpu().numpy(), 0, 2)
        label[0, i] = 1 - label[0, i]
        axs[i].imshow(o)
        axs[i].axis('off')
        s = np.copy(np.mean(o, axis=2))
        s.resize(200, 200)
        score_dic[f'niqe_{i}'].append(niqe(np.copy(s)))
#         score_dic[f'fid_{i}'].append(calculate_fid(np.moveaxis(img.cpu().numpy(), 1, 3), o.reshape((1, *o.shape)), False, 1))
    with torch.no_grad():
        o = g(img, label.cuda())
    score_dic[f'rec'].append(torch.nn.functional.mse_loss(img, o).data)
    plt.savefig(f'samples/cgan/{index:5d}.png')
    plt.close()

 30%|███       | 3/10 [00:02<00:06,  1.10it/s]

In [7]:
img, label = train_data[0]
img  = img.unsqueeze(0).cuda()
label = label.unsqueeze(0).cuda()
with torch.no_grad():
    o = g(img, torch.randint(0, 1, label.size()).cuda())
calculate_fid(np.moveaxis(img.cpu().numpy(), 1, 3), np.moveaxis(o.cpu().numpy(), 1, 3), False, 1)    

257.25740073027197

In [8]:
score_dic['rec']

[tensor(0.0330, device='cuda:0'),
 tensor(0.0188, device='cuda:0'),
 tensor(0.0163, device='cuda:0'),
 tensor(0.0113, device='cuda:0'),
 tensor(0.0293, device='cuda:0'),
 tensor(0.0340, device='cuda:0'),
 tensor(0.0383, device='cuda:0'),
 tensor(0.0147, device='cuda:0'),
 tensor(0.0171, device='cuda:0'),
 tensor(0.0088, device='cuda:0')]